<a href="https://colab.research.google.com/github/Crazytiger0527/ML_4_1/blob/main/ML_HW8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.24 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.9/78.2 GB disk)


In [ ]:
# ultralytics 라이브러리에서 YOLO 클래스를 임포트
from ultralytics import YOLO

# YOLO 모델 로드 (기본 YOLOv8 모델을 사용)
model = YOLO('yolov8n.pt')  # yolov8n, yolov8s, yolov8m, yolov8l, yolov8x 중 하나 선택

# 훈련에 사용할 데이터셋의 구성 파일 경로를 지정. Roboflow에서 YOLOv8 Format으로 Export하였기 때문에 바로 사용이 가능하다.
data_yaml_path = '/content/drive/MyDrive/Pokemon Detection.v1i.yolov8/data.yaml'

# 모델을 훈련 각 파라미터는 다음과 같다.
# data: 데이터셋 구성 파일 경로
#	epochs: 훈련할 에포크 수
#	imgsz: 입력 이미지의 크기
#	batch: 배치 크기
#	name: 훈련된 모델의 이름
model.train(data=data_yaml_path, epochs=100, imgsz=640, batch=16, name='custom_yolov8')

# 모델 평가 및 출력
results = model.val(data=data_yaml_path)
print(results)


In [ ]:
from ultralytics import YOLO
import cv2
import os
from matplotlib import pyplot as plt

# 훈련된 모델 가중치 가져오기
model = YOLO('/content/runs/detect/custom_yolov82/weights/best.pt')

# 테스트 이미지 경로 설정 및 결과 저장 폴더 지정
test_images_path = '/content/drive/MyDrive/Pokemon Detection.v1i.yolov8/test/images'
output_dir = 'test_predictions'
os.makedirs(output_dir, exist_ok=True)

# 테스트 이미지 로드
test_images = [os.path.join(test_images_path, img) for img in os.listdir(test_images_path) if img.endswith(('.png', '.jpg', '.jpeg'))]

# 전체 결과 저장 리스트 초기화
all_results = []

# 테스트 이미지 각각에 대해 예측을 수행하고, 예측 결과를 시각화하여 저장
for img_path in test_images:
    img = cv2.imread(img_path)  # 이미지 로드
    results = model.predict(img_path, save=False)  # 모델로 예측 수행
    annotated_img = results[0].plot()  # 예측 결과 시각화

    # 결과 저장
    output_img_path = os.path.join(output_dir, os.path.basename(img_path))  # 결과 이미지 저장 경로 설정
    cv2.imwrite(output_img_path, annotated_img)  # 결과 이미지 저장

    # 결과 저장 (전체 결과 리스트에 추가)
    all_results.append((os.path.basename(img_path), results[0]))

    # 결과 출력
    plt.figure(figsize=(10, 10))  # 출력 이미지 크기 설정
    plt.imshow(cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB))  # BGR을 RGB로 변환하여 출력
    plt.title(f'Predictions for {os.path.basename(img_path)}')  # 이미지 제목 설정
    plt.axis('off')  # 축 제거
    plt.show()  # 이미지 출력

# 테스트 데이터셋 평가
# 전체 테스트 데이터셋에 대해 모델 성능을 평가
results = model.val(data='/content/drive/MyDrive/Pokemon Detection.v1i.yolov8/data.yaml', split='test')
print(results)  # 평가 결과 출력

In [ ]:
from ultralytics import YOLO
import cv2
import os

# YOLO 모델 로드 (학습된 모델 사용)
model = YOLO('/content/runs/detect/custom_yolov8/weights/best.pt')  # 훈련된 모델 경로로 변경

# 동영상 파일 경로
input_video_path = '/content/drive/MyDrive/Pokemon Detection.v1i.yolov8/ddddddd.mp4'  # 입력 동영상 파일 경로
output_video_path = '/content/drive/MyDrive/Pokemon Detection.v1i.yolov8/output_video_with_tracking.mp4'  # 출력 동영상 파일 경로

# 동영상 파일 열기
cap = cv2.VideoCapture(input_video_path)

# 비디오 작성자 초기화
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 출력 동영상의 코덱 설정
out = None  # 비디오 작성자 초기화

# 프레임 처리
while cap.isOpened():  # 비디오 캡처가 열려있는 동안 반복
    ret, frame = cap.read()  # 한 프레임을 읽음
    if not ret:  # 프레임을 더 이상 읽을 수 없으면 반복 종료
        break

    # 객체 감지
    results = model.predict(frame, save=False)  # 모델을 사용하여 객체를 감지
    annotated_frame = results[0].plot()  # 예측 결과를 시각화하여 프레임에 그리기

    # 비디오 작성자 초기화 (첫 프레임에서)
    if out is None:  # 첫 번째 프레임에서만 초기화
        height, width, layers = annotated_frame.shape  # 프레임의 크기 가져오기
        out = cv2.VideoWriter(output_video_path, fourcc, 30, (width, height))  # 비디오 작성자 설정

    # 결과 프레임 저장
    out.write(annotated_frame)  # 주석이 추가된 프레임을 출력 동영상에 저장

# 자원 해제
cap.release()  # 비디오 캡처 자원 해제
out.release()  # 비디오 작성자 자원 해제

print(f'Output video saved to {output_video_path}')  # 출력 동영상 저장 경로 출력
